# Setup

In [13]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

## Potential Build Trajectories

We create every possible build trajectory for a given goal build. Builds are defined as a sorted set of integrant ids to be found in the resulting strain. The sorted set of tuples is known as a **part_hash**. We create a DAG with node ids being a sorted tuple of integrant ids and between these nodes indicating a transformation; plasmid_ids can be found as attributes along the edges.

*TODO* Weights will be determined by the availability of integrants
*TODO* start node should be provided. If a strain exists, an edge from START to the node should added with `weight=0`.

In [14]:
sess = aq.with_cache(timeout=70)

In [15]:
import networkx as nx
from typing import Hashable, Generator, TypeVar, List, Tuple, Optional, Dict, Callable
from pydent import AqSession
from pydent import ModelBase
from tqdm.auto import tqdm
import pandas as pd

Node = Hashable
_T = TypeVar('_T')


def is_leaf(n: Node, g: nx.DiGraph):
    successors = list(g.successors(n))
    if successors:
        return False
    return True


def is_root(n: Node, g: nx.DiGraph):
    predecessors = list(g.predecessors(n))
    if predecessors:
        return False
    return True


def iter_leaves(g: nx.DiGraph) -> Generator[Node, None, None]:
    for n in g.nodes():
        if is_leaf(n, g):
            yield n


def iter_roots(g: nx.DiGraph) -> Generator[Node, None, None]:
    for n in g.nodes():
        if is_root(n, g):
            yield n


def iter_split_sets(arr: List[_T]) -> Tuple[List[_T], _T]:
    for i in range(len(arr)):
        yield tuple(arr[:i] + arr[i + 1:]), arr[i]


class HashFunctions(object):

    @staticmethod
    def int_list_hash(ids: List[int], prefix: Optional[str] = None) -> Tuple[str, Tuple[int]]:
        return prefix, tuple(sorted(ids))


def group_by_sample_id(items):
    data = {}
    for item in items:
        data.setdefault(item.sample_id, list())
        data[item.sample_id].append(item)
    return data


def group_by_object_type_id(items):
    data = {}
    for item in items:
        data.setdefault(item.object_type_id, list())
        data[item.object_type_id].append(item)
    return data


def is_aq_model(x):
    return issubclass(x.__class__, ModelBase)


def get_models(sess, x, model):
    interface = sess.model_interface(model)
    if isinstance(x, int):
        res = [interface.find(x)]
        if res[0] is None:
            raise ValueError("Could not find {}".format(x))
    elif isinstance(x, str):
        res = [interface.find_by_name(x)]
        if res[0] is None:
            raise ValueError("Could not find {}".format(x))
    elif is_aq_model(x):
        res = [x]
        if res is None:
            raise ValueError
    elif isinstance(x, dict):
        res = interface.where(x)
        if res is None:
            raise ValueError
    elif isinstance(x, (list, tuple)):
        res = []
        for _x in x:
            res.extend(get_models(sess, _x, model))
    else:
        raise TypeError
    return res


class LIMSDB(object):
    YEAST_OBJECT_TYPES = ['Yeast Glycerol Stock', 'Yeast Plate', 'Yeast Competent Aliquot', 'Yeast Competent Cell']
    DNA_OBJECT_TYPES = ['Plasmid Stock', 'Plasmid Glycerol Stock', '1 ng/µL Plasmid Stock', 'Fragment Stock',
                        '1 ng/µL Fragment Stock']

    def __init__(self, sess: AqSession, strains=None, n=None, timeout: int = 60, progressbar: Callable = tqdm):
        """

        :param sess:
        :param strains:
        :param n:
        :param timeout:
        :param progressbar:
        """
        self.progressbar = progressbar
        self.sess = sess.with_cache(timeout=timeout)
        self.common = {
            'sample_types': {
                'yeast': self.sess.SampleType.find_by_name('Yeast Strain'),
                'plasmid': self.sess.SampleType.find_by_name('Plasmid'),
                'fragment': self.sess.SampleType.find_by_name('Fragment'),
            },
            'object_types': {
                'yeast': get_models(self.sess, self.YEAST_OBJECT_TYPES, 'ObjectType'),
                'dna': get_models(self.sess, self.DNA_OBJECT_TYPES, 'ObjectType')
            }
        }
        self.prov: nx.DiGraph = self.build_prov_graph(strains=strains, n=n)
        self.cache_items()

    def node_to_hash_fn(self, x, prefix=None):
        node, ndata = x
        return self.hash_fn(ndata['integrants'], prefix=prefix)

    def find_by_hash(self, x, prefix=None) -> List[Tuple[Node, Dict]]:
        k1 = self.hash_fn(x, prefix=prefix)
        visited = []
        for x in self.prov.nodes(data=True):

            ndata = x[1]
            if 'mating_type' in ndata:
                k2 = self.node_to_hash_fn(x, prefix=ndata['mating_type'])
                if k1 == k2:
                    visited.append(x)
        return visited

    def hash_fn(self, x, prefix=None):
        return HashFunctions.int_list_hash(x, prefix=prefix)

    def build_prov_graph(self, strains: Optional[List[ModelBase]] = None, n=None) -> nx.DiGraph:
        g = self._build_prov_graph(strains, n=n)
        grev = g.reverse()
        for n in grev.nodes():
            integrants = list(self._get_integrants(n, grev))
            g.nodes[n]['integrants'] = tuple(sorted(integrants))
        return g

    @staticmethod
    def _get_integrants(n, grev):
        for n1, n2 in list(nx.dfs_edges(grev, n)):
            edata = grev[n1][n2]
            if edata['field_name'] == 'Integrant':
                yield n2

    def _build_prov_graph(self, strains: Optional[List[ModelBase]] = None, n=None) -> nx.DiGraph:
        g = nx.DiGraph()

        def add_sample_node(s, **kwargs):
            g.add_node(s.id, sample_type=s.sample_type_id, sample=s.dump(), **kwargs)

        if not strains:
            assert n
            strains = self.sess.Sample.last(n, query={
                'sample_type_id': self.common['sample_types']['yeast'].id,
                'user_id': 66
            })
        while strains:
            if strains:
                self.sess.browser.get(strains, {
                    'field_values': {
                        'field_type': 'allowable_field_types',
                        'sample': []
                    },
                    'sample_type': {
                        'field_types': 'allowable_field_types'
                    }
                })

                parents = {}
                for s in strains:
                    add_sample_node(s, mating_type=s.properties['Mating Type'])

                    parent = None
                    if 'Parent' in s.properties:
                        parent = s.properties['Parent']

                    integrant = None
                    if 'Integrant' in s.properties:
                        integrant = s.properties['Integrant']

                    if parent is not None:
                        parents[parent.id] = parent
                        add_sample_node(parent, mating_type=parent.properties['Mating Type'])
                        g.add_edge(parent.id, s.id, field_name='Parent')

                    if integrant is not None:
                        add_sample_node(integrant)
                        g.add_edge(integrant.id, s.id, field_name='Integrant', sample=integrant.dump())

            new_parents = {}
            sids = [s.id for s in strains]
            for sid, p in parents.items():
                if sid not in sids:
                    new_parents[sid] = p
            strains = list(new_parents.values())
        return g

    def find_in_cache(self, query: dict, model_name: str):
        visited = []
        for item in self.sess.browser.model_cache[model_name].values():
            for k, v in query.items():
                if getattr(item, k) == v:
                    visited.append(item)
        return visited

    def find_yeast_items(self, sample_id):
        items = self.find_items(sample_id, [ot.id for ot in self.common['object_types']['yeast']])
        return items

    def find_dna_items(self, sample_id):
        items = self.find_items(sample_id, [ot.id for ot in self.common['object_types']['dna']])
        return items

    def find_items(self, sample_id, otids):
        visited = []
        for item in self.sess.browser.model_cache["Item"].values():
            if item.sample_id == sample_id and item.location != 'deleted' and item.object_type_id in otids:
                visited.append(item)
        return visited

    def cache_items(self):
        all_ots = []
        for k, v in self.common['object_types'].items():
            all_ots.extend(v)
        ot_by_st = {}
        for ot in all_ots:
            ot_by_st.setdefault(ot.sample_type_id, list())
            ot_by_st[ot.sample_type_id].append(ot)

        for stid, ots in ot_by_st.items():
            samples = self.find_in_cache({'sample_type_id': stid}, 'Sample')
            self._cache_items([s.id for s in samples], ots)

    def _cache_items(self, sample_ids: List[int], object_types):
        sess = self.sess
        ots = get_models(sess, object_types, 'ObjectType')
        items = []
        for ot in self.progressbar(ots):
            query = {'sample_id': sample_ids, 'object_type_id': ot.id}
            items_ = sess.Item.where(query)
            items_ = [i for i in items_ if i.location != 'deleted']
            items.extend(items_)
        return items

    def create_trajectories(self, prefix, parts_arr, g=None) -> nx.DiGraph:
        """
        Create all possible build trajectories.

        :param prefix:
        :param parts_arr:
        :param g:
        :return:
        """
        if g is None:
            g = nx.DiGraph()
        for parts_arr_ in parts_arr:
            n1 = self.hash_fn(prefix, parts_arr_)
            visited = []
            for parts, x in iter_split_sets(parts_arr_):
                if len(parts) > 1:
                    visited.append(parts)
                n2 = self.hash_fn(prefix, parts)
                g.add_edge(n2, n1, plasmid_id=x)
            self.create_trajectories(prefix, visited, g=g)
        return g


def by_key_value(arr, keyfn, valuefn, iffn=None):
    data = {}
    for a in arr:
        key = keyfn(a)
        data.setdefault(key, list())
        v = valuefn(a)
        if iffn and iffn(v):
            data[key].append(valuefn(a))
    return data


from copy import deepcopy


class BuildPaths(object):

    def __init__(self, db: LIMSDB):
        self.db = db

    def create_trajectories(self, prefix, parts_arr, g=None) -> nx.DiGraph:
        """
        Create all possible build trajectories.

        :param prefix:
        :param parts_arr:
        :param g:
        :return:
        """
        hash_fn = self.db.hash_fn
        if g is None:
            g = nx.DiGraph()
        for parts_arr_ in parts_arr:
            n1 = hash_fn(x=parts_arr_, prefix=prefix)
            visited = []
            for parts, x in iter_split_sets(parts_arr_):
                if len(parts) > 1:
                    visited.append(parts)
                n2 = hash_fn(x=parts, prefix=prefix)
                g.add_edge(n2, n1, plasmid_id=x)
            self.create_trajectories(prefix, visited, g=g)
        return g

    @staticmethod
    def validate_parts(parts):

        for part in parts:
            assert 'gate' in part
            assert 'haploid' in part
            assert 'sample' in part

    def generate_build_paths(self, parts: List[dict]):
        parts = deepcopy(parts)
        for part in parts:
            sample = self.db.sess.Sample.find_by_name(part['sample']['name'])
            part['sample'] = sample.dump()
        self.validate_parts(parts)
        partsdict = by_key_value(parts, lambda x: x['haploid'], lambda x: x['sample']['id'], lambda x: True)
        trajectories = {}
        print(partsdict)
        for haploid, parts in partsdict.items():
            traj = self.create_trajectories(prefix=haploid, parts_arr=[parts])

            nodelist = list(traj.nodes())
            traj.add_node("START")

            for n1 in nodelist:
                prefix, parts_ = n1
                if prefix == 'Mat A':
                    prefix = 'MATa'
                elif prefix == 'Mat Alpha':
                    prefix = 'MATalpha'
                else:
                    raise RuntimeError('prefix "{}" unexpected for {}'.format(prefix, n1))

                found = self.db.find_by_hash(parts_, prefix=prefix)
                for sample_id, ndata2 in found:
                    items = self.db.find_yeast_items(sample_id)
                    if items:
                        traj.add_edge('START', n1, weight=0)

            for n1, n2, edata in traj.edges(data=True):
                if n1 != 'START':
                    plasmid_id = edata['plasmid_id']
                    items = self.db.find_dna_items(plasmid_id)
                    if items:
                        edata['weight'] = 1
                    else:
                        edata['weight'] = 4

            trajectories[haploid] = traj
        return trajectories


def by_key_value(arr, keyfn, valuefn, iffn=None):
    data = {}
    for a in arr:
        key = keyfn(a)
        data.setdefault(key, list())
        v = valuefn(a)
        if iffn and iffn(v):
            data[key].append(valuefn(a))
    return data

class BuildPaths(object):

    def __init__(self, db: LIMSDB):
        self.db = db

    def create_trajectories(self, prefix, parts_arr, g=None) -> nx.DiGraph:
        """
        Create all possible build trajectories.

        :param prefix:
        :param parts_arr:
        :param g:
        :return:
        """
        hash_fn = self.db.hash_fn
        if g is None:
            g = nx.DiGraph()
        for parts_arr_ in parts_arr:
            n1 = hash_fn(x=parts_arr_, prefix=prefix)
            visited = []
            for parts, x in iter_split_sets(parts_arr_):
                if len(parts) > 1:
                    visited.append(parts)
                n2 = hash_fn(x=parts, prefix=prefix)
                g.add_edge(n2, n1, plasmid_id=x)
            self.create_trajectories(prefix, visited, g=g)
        return g

    @staticmethod
    def validate_parts(parts):

        for part in parts:
            assert 'gate' in part
            assert 'haploid' in part
            assert 'sample' in part



    def generate_build_paths(self, parts: List[dict]):
        parts = deepcopy(parts)
        for part in parts:
            sample = self.db.sess.Sample.find_by_name(part['sample']['name'])
            part['sample'] = sample.dump()
        self.validate_parts(parts)
        partsdict = by_key_value(parts, lambda x: x['haploid'], lambda x: x['sample']['id'], lambda x: True)
        trajectories = {}
        print(partsdict)
        for haploid, parts in partsdict.items():
            traj = self.create_trajectories(prefix=haploid, parts_arr=[parts])

            nodelist = list(traj.nodes())
            traj.add_node("START")

            for n1 in nodelist:
                prefix, parts_ = n1
                if prefix == 'Mat A':
                    prefix = 'MATa'
                elif prefix == 'Mat Alpha':
                    prefix = 'MATalpha'
                else:
                    raise RuntimeError('prefix "{}" unexpected for {}'.format(prefix, n1))

                found = self.db.find_by_hash(parts_, prefix=prefix)
                for sample_id, ndata2 in found:
                    items = self.db.find_yeast_items(sample_id)
                    if items:
                        traj.add_edge('START', n1, weight=0)

            for n1, n2, edata in traj.edges(data=True):
                if n1 != 'START':
                    plasmid_id = edata['plasmid_id']
                    items = self.db.find_dna_items(plasmid_id)
                    if items:
                        edata['weight'] = 1
                    else:
                        edata['weight'] = 4

            trajectories[haploid] = traj
        return trajectories

    def generate_instructions(self, parts):
        self.validate_parts(parts)

        instructions = []

        for h, g in self.generate_build_paths(parts).items():
            leaves = list(iter_leaves(g))
            assert len(leaves) == 1
            paths = list(nx.all_shortest_paths(g, 'START', leaves[0]))
            path_weights = []
            for path in paths:
                weights = []
                for n1, n2 in nx.utils.pairwise(path):
                    weights.append(g[n1][n2]['weight'])
                path_weights.append(tuple(weights))
            sorted_paths = sorted(list(zip(paths, path_weights)), key=lambda x: x[1])
            best_path = sorted_paths[0]
            for i, (n1, n2) in enumerate(nx.utils.pairwise(best_path[0])):
                if n1 == "START":
                    continue
                p1 = n1[0]
                p2 = n2[0]
                _prefix = {
                    'Mat A': 'MATa',
                    'Mat Alpha': 'MATalpha'
                }
                p1 = _prefix[p1]
                p2 = _prefix[p2]

                s1 = self.db.find_by_hash(list(n1[1]), prefix=p1)
                s2 = self.db.find_by_hash(list(n2[1]), prefix=p2)
                edata = g[n1][n2]

                if s1:
                    parent = [(_s[0], _s[1]['sample']['name']) for _s in s1]
                else:
                    parent = n1

                if s2:
                    result = [(_s[0], _s[1]['sample']['name']) for _s in s2]
                else:
                    result = n2

                instructions.append({
                    'step': i,
                    'parent': parent,
                    'dna': edata['plasmid_id'],
                    'result': result,
                    'weight': edata['weight']
                })

        return instructions

    def parse_builds(self, builds):
        all_rows = []
        for build in builds:
            parts = build['parts']
            data = dict(build)
            data['id'] = data['name'] + "_" + str(data['permutation'])
            del data['parts']
            rows = self.generate_instructions(parts)
            for row in rows:
                row.update(data)
            all_rows.extend(rows)
        return pd.DataFrame(all_rows)

## Generate LIMS

In [16]:
db = LIMSDB(sess, n=300)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

### Generate Build Paths

In [6]:
import json 

with open('build_permutations_2021-03-08T10:16:09.773080-08:00_corrected.json', 'r') as f:
    builds = json.load(f)

buildpaths = BuildPaths(db)
builddf = buildpaths.parse_builds(builds)

{'Mat A': [33579, 1380, 36332, 36333], 'Mat Alpha': [1096, 5533, 36334, 36335, 36336]}
{'Mat A': [33579, 1380, 36332, 36337], 'Mat Alpha': [1096, 5533, 36334, 36335, 36336]}
{'Mat A': [33579, 1380, 36332, 36337], 'Mat Alpha': [1096, 5533, 36338, 36335, 36336]}
{'Mat A': [33579, 1380, 36339, 36340], 'Mat Alpha': [1096, 5534, 36341, 36342, 36343]}
{'Mat A': [33579, 1380, 36339, 36344], 'Mat Alpha': [1096, 5534, 36341, 36342, 36343]}
{'Mat A': [33579, 1380, 36339, 36344], 'Mat Alpha': [1096, 5534, 36345, 36342, 36343]}
{'Mat A': [33579, 1380, 36332, 36333], 'Mat Alpha': [1096, 5533, 36346, 36347]}
{'Mat A': [33579, 1380, 36332, 36337], 'Mat Alpha': [1096, 5533, 36346, 36347]}
{'Mat A': [33579, 1380, 36332, 36337], 'Mat Alpha': [1096, 5533, 36348, 36347]}
{'Mat A': [33579, 1380, 36339, 36349], 'Mat Alpha': [1096, 5534, 36344, 36350]}
{'Mat A': [33579, 1380, 36339, 36351], 'Mat Alpha': [1096, 5534, 36344, 36350]}
{'Mat A': [33579, 1380, 36339, 36351], 'Mat Alpha': [1096, 5534, 36352, 36350]

In [677]:
# builddf.to_clipboard()

### Display Instructions

In [9]:
from IPython import display

df = builddf[builddf.permutation == 0]

df['dna_name'] = [aq.Sample.find(x).name for x in df.dna]

df.to_csv("build_plan.csv")

display.display(df[df.step == 1])

fragments = {}
for _, row in df[df.step == 1].iterrows():
    fragments.setdefault(row.dna, 0)
    fragments[row.dna] += 1

num_transformations = len(df[df.step == 1])
print("Number of transformations: " + str(num_transformations))

print('Fragment Usage')
display.display(fragments)

print("Fragment Locations")
item_rows = []
for fid in fragments:
    f = db.sess.Sample.find(fid)
    items = db.find_dna_items(fid)
    for item in items:
        row = {
            'sample_name': f.name,
            'sample_id': f.id,
            'item_id': item.id,
            'type': item.object_type.name,
            'location': item.location
        }
        item_rows.append(row)
    if not items:
        row = {
            'sample_name': f.name,
            'sample_id': f.id,
            'item_id': "no item",
            'type': None,
            'location': 'no item'
        }
        item_rows.append(row)
item_df = pd.DataFrame(item_rows)
display.display(item_df)


item_rows = []
parent_ids = {}
for _, row in df[df.step == 1].iterrows():
    parents = row.parent
    parents = [p for p in parents if 'TRASH' not in p[1]]
    assert len(parents) == 1
    parent_id = parents[0][0]
    parent_ids.setdefault(parent_id, 0)
    parent_ids[parent_id] += 1
    
print('Yeast Usage')
display.display(parent_ids)
    
print("Yeast Locations")
yeast_rows = []
for sid in parent_ids:
    sample = db.sess.Sample.find(sid)
    items = db.find_yeast_items(sid)
    for item in items:
        row = {
            'sample_name': sample.name,
            'sample_id': sample.id,
            'item_id': item.id,
            'type': item.object_type.name,
            'location': item.location
        }
        yeast_rows.append(row)
    if not items:
        row = {
            'sample_name': sample.name,
            'sample_id': sample.id,
            'item_id': "no item",
            'type': 'none',
            'location': 'no item'
        }
        yeast_rows.append(row)
yeast_df = pd.DataFrame(yeast_rows)
display.display(yeast_df)

<ipython-input-9-8a60a5055ea1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dna_name'] = [aq.Sample.find(x).name for x in df.dna]


,step,parent,dna,result,weight,name,permutation,created_at,id,dna_name
0,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36332,"[(36398, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>
2,1,"[(24943, CEN.PK2 - MAT alpha ::: pMODKan-HO-pA...",36336,"(Mat Alpha, (1096, 5533, 36336))",4,DSGRN NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (BAD)_0,URA-URA<pGRR-W20W36-yeGFP>
15,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36339,"[(36397, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
17,1,"[(24942, CEN.PK2 - MAT alpha ::: pMODKan-HO-pA...",36343,"(Mat Alpha, (1096, 5534, 36343))",4,DSGRN NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN NOR (GOOD)_0,URA-URA<pGRR-W10W20-yeGFP>
30,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36332,"[(36398, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,DSGRN OR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN OR (BAD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>
32,1,"[(24943, CEN.PK2 - MAT alpha ::: pMODKan-HO-pA...",36347,"[(36379, CEN.PK2 - MAT alpha ::: pMODKan-HO-pA...",1,DSGRN OR (BAD),0,2021-03-08T09:30:22.723934-08:00,DSGRN OR (BAD)_0,URA-URA<pGRR-W5W36-yeGFP>
42,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36339,"[(36397, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,DSGRN OR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN OR (GOOD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>
44,1,"[(24942, CEN.PK2 - MAT alpha ::: pMODKan-HO-pA...",36350,"(Mat Alpha, (1096, 5534, 36350))",4,DSGRN OR (GOOD),0,2021-03-08T09:30:22.723934-08:00,DSGRN OR (GOOD)_0,URA-URA<pGRR-W10W17-yeGFP>
54,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36353,"[(36396, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,GANDER NOR (BAD),0,2021-03-08T09:30:22.723934-08:00,GANDER NOR (BAD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W5>
57,1,"[(33811, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",36339,"[(36397, CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1...",1,GANDER NOR (GOOD),0,2021-03-08T09:30:22.723934-08:00,GANDER NOR (GOOD)_0,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>


Number of transformations: 14
Fragment Usage


{36332: 3,
 36336: 1,
 36339: 4,
 36343: 1,
 36347: 1,
 36350: 1,
 36353: 1,
 36357: 1,
 36359: 1}

Fragment Locations


,sample_name,sample_id,item_id,type,location
0,HIS-HIS<7XTetO-pMinCyc1-URGR-W8>,36332,512002,1 ng/µL Fragment Stock,M20.24.1.40
1,URA-URA<pGRR-W20W36-yeGFP>,36336,no item,None,no item
2,HIS-HIS<7XTetO-pMinCyc1-URGR-W34>,36339,511753,1 ng/µL Fragment Stock,M20.23.12.2
3,URA-URA<pGRR-W10W20-yeGFP>,36343,no item,None,no item
4,URA-URA<pGRR-W5W36-yeGFP>,36347,512004,1 ng/µL Fragment Stock,M20.24.1.42
5,URA-URA<pGRR-W10W17-yeGFP>,36350,no item,None,no item
6,HIS-HIS<7XTetO-pMinCyc1-URGR-W5>,36353,511754,1 ng/µL Fragment Stock,M20.23.12.5
7,URA-URA<pGRR-W36W36-yeGFP>,36357,512402,1 ng/µL Fragment Stock,M20.24.1.32
8,URA-URA<pGRR-W17W17-yeGFP>,36359,512003,1 ng/µL Fragment Stock,M20.24.1.41


Yeast Usage


{33811: 8, 24943: 2, 24942: 3, 24940: 1}

Yeast Locations


,sample_name,sample_id,item_id,type,location
0,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,451322,Yeast Glycerol Stock,M80.7.8.56
1,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,449991,Yeast Plate,DFP.6.3.7
2,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,451421,Yeast Competent Cell,M80C.2.15.9
3,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,451422,Yeast Competent Cell,M80C.2.15.75
4,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,451423,Yeast Competent Cell,M80C.2.15.76
5,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,512162,Yeast Competent Cell,M80C.3.0.26
6,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,512163,Yeast Competent Cell,M80C.3.0.64
7,CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-H...,33811,512164,Yeast Competent Cell,M80C.3.1.17
8,CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 ...,24943,153612,Yeast Glycerol Stock,M80.5.8.56
9,CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 ...,24943,315706,Yeast Competent Cell,M80C.0.2.43


In [26]:
transformations = []
for _, row in df.iterrows():
    if isinstance(row.parent, list):
        parent_id = row.parent[0][0]
    else:
        parent_id = row.parent
    dna_id = row.dna
    transformation = (parent_id, dna_id)
    transformations.append(transformation)
    
df['transformations'] = transformations

<ipython-input-26-a1144f363ee3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transformations'] = transformations


In [41]:
todays_transformations = list(set(df[df['step'] == 1].transformations))

rows = []
for parent_id, plasmid_id in todays_transformations:
    parent = aq.Sample.find(parent_id)

    plasmid = aq.Sample.find(plasmid_id)
    alt_plasmid = aq.Sample.find_by_name(plasmid.name + ' (v2)')
    if alt_plasmid:
        plasmid = alt_plasmid
        
    print(parent_id, ",", plasmid.id)
    
    items = aq.Item.where({'sample_id': plasmid.id, 'object_type_id': aq.ObjectType.find_by_name("Fragment Stock").id})
    items = [i for i in items if i.location != 'deleted']
    
    
    rows.append({
        'parent': parent_id,
        'dna': plasmid.id,
        'items': [(item.id, item.location) for item in items]
    })
    
pd.DataFrame(rows).to_clipboard()
#     print(plasmid.name)

24943 , 36445
33811 , 36353
24943 , 36448
24942 , 36451
24940 , 36447
24942 , 36446
24942 , 36450
33811 , 36339
33811 , 36332


Unverified PCR Fragment
M20.23.13.67
__Part
Part of Collection
__Part
Part of Collection
__Part
Part of Collection
Gel Slice
deleted
Fragment Stock
M20.24.1.31
